In [1]:
#!nvidia-smi
#using a GeForce GTX1080 Ti for reproducibility for all timing experiments

In [2]:
import torch
import torchvision
from torch import nn, optim, autograd
from torch.nn import functional as F
import numpy as np
from sklearn.metrics import roc_auc_score
import scipy
from utils.LB_utils import * 
from utils.load_not_MNIST import notMNIST
import os
import time
import matplotlib.pyplot as plt
from laplace import Laplace
import utils.scoring as scoring

s = 1
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cuda_status = torch.cuda.is_available()
print("device: ", device)
print("cuda status: ", cuda_status)

device:  cuda
cuda status:  True


In [4]:
### define network
class ConvNet(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.net = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.Conv2d(16, 32, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.Flatten(),
            torch.nn.Linear(4 * 4 * 32, num_classes)
        )
    def forward(self, x):
        out = self.net(x)
        return out

In [5]:
BATCH_SIZE_TRAIN_MNIST = 128
BATCH_SIZE_TEST_MNIST = 128
MAX_ITER_MNIST = 6
LR_TRAIN_MNIST = 10e-6

In [6]:
MNIST_transform = torchvision.transforms.ToTensor()

MNIST_train = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=True,
        download=True,
        transform=MNIST_transform)

MNIST_train_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_train,
    batch_size=BATCH_SIZE_TRAIN_MNIST,
    shuffle=True
)


MNIST_test = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform)

MNIST_test_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_test,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False,
)

In [7]:
mnist_model = ConvNet().to(device)
loss_function = torch.nn.CrossEntropyLoss()

mnist_train_optimizer = torch.optim.Adam(mnist_model.parameters(), lr=1e-3, weight_decay=5e-4)
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer_s{}.pth".format(s)

In [8]:
#Training routine

def train(model, train_loader, optimizer, max_iter, path, verbose=True):
    max_len = len(train_loader)

    for iter in range(max_iter):
        for batch_idx, (x, y) in enumerate(train_loader):
            
            x, y = x.to(device), y.to(device)
            
            output = model(x)

            accuracy = get_accuracy(output, y)

            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if verbose and batch_idx % 50 == 0:
                print(
                    "Iteration {}; {}/{} \t".format(iter, batch_idx, max_len) +
                    "Minibatch Loss %.3f  " % (loss) +
                    "Accuracy %.0f" % (accuracy * 100) + "%"
                )

    print("saving model at: {}".format(path))
    torch.save(mnist_model.state_dict(), path)

In [9]:
#train(mnist_model, MNIST_train_loader, mnist_train_optimizer, MAX_ITER_MNIST, MNIST_PATH, verbose=True)

In [10]:
#predict in distribution
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer_s{}.pth".format(s)
#MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer.pth"

mnist_model = ConvNet().to(device)
print("loading model from: {}".format(MNIST_PATH))
mnist_model.load_state_dict(torch.load(MNIST_PATH))
mnist_model.eval()

acc = []

max_len = len(MNIST_test_loader)
for batch_idx, (x, y) in enumerate(MNIST_test_loader):

    x, y = x.to(device), y.to(device)
    output = mnist_model(x)

    accuracy = get_accuracy(output, y)
    if batch_idx % 10 == 0:
        print(
            "Batch {}/{} \t".format(batch_idx, max_len) + 
            "Accuracy %.0f" % (accuracy * 100) + "%"
        )
    acc.append(accuracy)

avg_acc = np.mean(acc)
print('overall test accuracy on MNIST: {:.02f} %'.format(avg_acc * 100))


loading model from: pretrained_weights/MNIST_pretrained_10_classes_last_layer_s1.pth
Batch 0/79 	Accuracy 100%
Batch 10/79 	Accuracy 96%
Batch 20/79 	Accuracy 98%
Batch 30/79 	Accuracy 98%
Batch 40/79 	Accuracy 100%
Batch 50/79 	Accuracy 99%
Batch 60/79 	Accuracy 100%
Batch 70/79 	Accuracy 98%
overall test accuracy on MNIST: 98.84 %


In [11]:
BATCH_SIZE_TEST_FMNIST = 128
BATCH_SIZE_TEST_KMNIST = 128

In [12]:
FMNIST_test = torchvision.datasets.FashionMNIST(
        '~/data/fmnist', train=False, download=True,
        transform=MNIST_transform)   #torchvision.transforms.ToTensor())

FMNIST_test_loader = torch.utils.data.DataLoader(
    FMNIST_test,
    batch_size=BATCH_SIZE_TEST_FMNIST, shuffle=False)

In [13]:
KMNIST_test = torchvision.datasets.KMNIST(
        '~/data/kmnist', train=False, download=True,
        transform=MNIST_transform)

KMNIST_test_loader = torch.utils.data.DataLoader(
    KMNIST_test,
    batch_size=BATCH_SIZE_TEST_KMNIST, shuffle=False)

In [14]:
#root = os.path.abspath('~/data')
root = os.path.expanduser('~/data')

# Instantiating the notMNIST dataset class we created
notMNIST_test = notMNIST(root=os.path.join(root, 'notMNIST_small'),
                               transform=MNIST_transform)

# Creating a dataloader
notMNIST_test_loader = torch.utils.data.dataloader.DataLoader(
                            dataset=notMNIST_test,
                            batch_size=BATCH_SIZE_TEST_KMNIST,
                            shuffle=False)

File F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png is broken
File A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png is broken


# MAP estimate

In [15]:
targets = MNIST_test.targets.numpy()
targets_FMNIST = FMNIST_test.targets.numpy()
targets_notMNIST = notMNIST_test.targets.numpy().astype(int)
targets_KMNIST = KMNIST_test.targets.numpy()

In [16]:
MNIST_test_in_MAP = predict_MAP(mnist_model, MNIST_test_loader, device=device).cpu().numpy()
MNIST_test_out_FMNIST_MAP = predict_MAP(mnist_model, FMNIST_test_loader, device=device).cpu().numpy()
MNIST_test_out_notMNIST_MAP = predict_MAP(mnist_model, notMNIST_test_loader, device=device).cpu().numpy()
MNIST_test_out_KMNIST_MAP = predict_MAP(mnist_model, KMNIST_test_loader, device=device).cpu().numpy()

In [17]:
# compute average log-likelihood for Diag
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_MAP)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_MAP)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_MAP)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_MAP)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

0.03344982489943504
4.6731367111206055
6.672377586364746
6.748490810394287


In [18]:
#compute the Expected confidence estimate
print(scoring.expected_calibration_error(targets, MNIST_test_in_MAP))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_MAP))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_MAP))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_MAP))

0.009983010101010131
0.5903608484848487
0.6404300589747076
0.6294307171717172


In [20]:
##FPR95
print(get_fpr95(MNIST_test_in_MAP, MNIST_test_out_FMNIST_MAP))

(0.1311, 0.9366349875926971)


In [ ]:
acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP = get_in_dist_values(MNIST_test_in_MAP, targets)
acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP = get_out_dist_values(MNIST_test_in_MAP, MNIST_test_out_FMNIST_MAP, targets_FMNIST)
acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP = get_out_dist_values(MNIST_test_in_MAP, MNIST_test_out_notMNIST_MAP, targets_notMNIST)
acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP = get_out_dist_values(MNIST_test_in_MAP, MNIST_test_out_KMNIST_MAP, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP, 'MNIST', 'MAP')
print_out_dist_values(acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP, 'MNIST', test='FMNIST', method='MAP')
print_out_dist_values(acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP, 'MNIST', test='notMNIST', method='MAP')
print_out_dist_values(acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP, 'MNIST', test='KMNIST', method='MAP')

In [ ]:
num_samples = 100

# Diag Hessian Sampling estimate

In [ ]:
la_diag = Laplace(mnist_model, 'classification', 
                     subset_of_weights='last_layer', 
                     hessian_structure='diag',
                     prior_precision=1e-0) # 5e-4 # Choose prior precision according to weight decay
la_diag.fit(MNIST_train_loader)

In [ ]:
MNIST_test_in_D = predict_samples(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_D = predict_samples(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_D = predict_samples(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_D = predict_samples(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for Diag
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_D)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_D)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_D)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_D)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
#compute the Expected confidence estimate
print(scoring.expected_calibration_error(targets, MNIST_test_in_D))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_D))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_D))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_D))

In [ ]:
acc_in_D, prob_correct_in_D, ent_in_D, MMC_in_D = get_in_dist_values(MNIST_test_in_D, targets)
acc_out_FMNIST_D, prob_correct_out_FMNIST_D, ent_out_FMNIST_D, MMC_out_FMNIST_D, auroc_out_FMNIST_D = get_out_dist_values(MNIST_test_in_D, MNIST_test_out_FMNIST_D, targets_FMNIST)
acc_out_notMNIST_D, prob_correct_out_notMNIST_D, ent_out_notMNIST_D, MMC_out_notMNIST_D, auroc_out_notMNIST_D = get_out_dist_values(MNIST_test_in_D, MNIST_test_out_notMNIST_D, targets_notMNIST)
acc_out_KMNIST_D, prob_correct_out_KMNIST_D, ent_out_KMNIST_D, MMC_out_KMNIST_D, auroc_out_KMNIST_D = get_out_dist_values(MNIST_test_in_D, MNIST_test_out_KMNIST_D, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_D, prob_correct_in_D, ent_in_D, MMC_in_D, 'MNIST', 'Diag')
print_out_dist_values(acc_out_FMNIST_D, prob_correct_out_FMNIST_D, ent_out_FMNIST_D, MMC_out_FMNIST_D, auroc_out_FMNIST_D, 'MNIST', test='fmnist', method='Diag')
print_out_dist_values(acc_out_notMNIST_D, prob_correct_out_notMNIST_D, ent_out_notMNIST_D, MMC_out_notMNIST_D, auroc_out_notMNIST_D, 'MNIST', test='notMNIST', method='Diag')
print_out_dist_values(acc_out_KMNIST_D, prob_correct_out_KMNIST_D, ent_out_KMNIST_D, MMC_out_KMNIST_D, auroc_out_KMNIST_D, 'MNIST', test='KMNIST', method='Diag')

# KFAC Laplace Approximation (sampling)

In [ ]:
la_kron = Laplace(mnist_model, 'classification', 
                     subset_of_weights='last_layer', 
                     hessian_structure='kron',
                     prior_precision=5e-0) # 5e-4 # Choose prior precision according to weight decay
la_kron.fit(MNIST_train_loader)

In [ ]:
MNIST_test_in_KFAC = predict_samples(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_KFAC = predict_samples(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_KFAC = predict_samples(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_KFAC = predict_samples(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_KFAC)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_KFAC)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_KFAC)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_KFAC)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
# compute ECE for KFAC
print(scoring.expected_calibration_error(targets, MNIST_test_in_KFAC))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_KFAC))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_KFAC))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_KFAC))

In [ ]:
acc_in_KFAC, prob_correct_in_KFAC, ent_in_KFAC, MMC_in_KFAC = get_in_dist_values(MNIST_test_in_KFAC, targets)
acc_out_FMNIST_KFAC, prob_correct_out_FMNIST_KFAC, ent_out_FMNIST_KFAC, MMC_out_FMNIST_KFAC, auroc_out_FMNIST_KFAC = get_out_dist_values(MNIST_test_in_KFAC, MNIST_test_out_FMNIST_KFAC, targets_FMNIST)
acc_out_notMNIST_KFAC, prob_correct_out_notMNIST_KFAC, ent_out_notMNIST_KFAC, MMC_out_notMNIST_KFAC, auroc_out_notMNIST_KFAC = get_out_dist_values(MNIST_test_in_KFAC, MNIST_test_out_notMNIST_KFAC, targets_notMNIST)
acc_out_KMNIST_KFAC, prob_correct_out_KMNIST_KFAC, ent_out_KMNIST_KFAC, MMC_out_KMNIST_KFAC, auroc_out_KMNIST_KFAC = get_out_dist_values(MNIST_test_in_KFAC, MNIST_test_out_KMNIST_KFAC, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_KFAC, prob_correct_in_KFAC, ent_in_KFAC, MMC_in_KFAC, 'MNIST', 'KFAC')
print_out_dist_values(acc_out_FMNIST_KFAC, prob_correct_out_FMNIST_KFAC, ent_out_FMNIST_KFAC, MMC_out_FMNIST_KFAC, auroc_out_FMNIST_KFAC, 'MNIST', test='fmnist', method='KFAC')
print_out_dist_values(acc_out_notMNIST_KFAC, prob_correct_out_notMNIST_KFAC, ent_out_notMNIST_KFAC, MMC_out_notMNIST_KFAC, auroc_out_notMNIST_KFAC, 'MNIST', test='notMNIST', method='KFAC')
print_out_dist_values(acc_out_KMNIST_KFAC, prob_correct_out_KMNIST_KFAC, ent_out_KMNIST_KFAC, MMC_out_KMNIST_KFAC, auroc_out_KMNIST_KFAC, 'MNIST', test='KMNIST', method='KFAC')

# Laplace Bridge (Diagonal)

In [ ]:
MNIST_test_in_LB_D = predict_LB(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_LB_D = predict_LB(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_LB_D = predict_LB(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_LB_D = predict_LB(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# # compute average log-likelihood for LB
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_LB_D)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_LB_D)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_LB_D)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_LB_D)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
#compute ECE for LB
print(scoring.expected_calibration_error(targets, MNIST_test_in_LB_D))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_LB_D))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_LB_D))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_LB_D))

In [ ]:
acc_in_LB_D, prob_correct_in_LB_D, ent_in_LB_D, MMC_in_LB_D = get_in_dist_values(MNIST_test_in_LB_D, targets)
acc_out_FMNIST_LB_D, prob_correct_out_FMNIST_LB_D, ent_out_FMNIST_LB_D, MMC_out_FMNIST_LB_D, auroc_out_FMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_D, MNIST_test_out_FMNIST_LB_D, targets_FMNIST)
acc_out_notMNIST_LB_D, prob_correct_out_notMNIST_LB_D, ent_out_notMNIST_LB_D, MMC_out_notMNIST_LB_D, auroc_out_notMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_D, MNIST_test_out_notMNIST_LB_D, targets_notMNIST)
acc_out_KMNIST_LB_D, prob_correct_out_KMNIST_LB_D, ent_out_KMNIST_LB_D, MMC_out_KMNIST_LB_D, auroc_out_KMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_D, MNIST_test_out_KMNIST_LB_D, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_LB_D, prob_correct_in_LB_D, ent_in_LB_D, MMC_in_LB_D, 'MNIST', 'LB_D')
print_out_dist_values(acc_out_FMNIST_LB_D, prob_correct_out_FMNIST_LB_D, ent_out_FMNIST_LB_D, MMC_out_FMNIST_LB_D, auroc_out_FMNIST_LB_D, 'MNIST', test='fmnist', method='LB_D')
print_out_dist_values(acc_out_notMNIST_LB_D, prob_correct_out_notMNIST_LB_D, ent_out_notMNIST_LB_D, MMC_out_notMNIST_LB_D, auroc_out_notMNIST_LB_D, 'MNIST', test='notMNIST', method='LB_D')
print_out_dist_values(acc_out_KMNIST_LB_D, prob_correct_out_KMNIST_LB_D, ent_out_KMNIST_LB_D, MMC_out_KMNIST_LB_D, auroc_out_KMNIST_LB_D, 'MNIST', test='KMNIST', method='LB_D')

# LB diag norm

In [ ]:
MNIST_test_in_LB_Dn = predict_LB_norm(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_LB_Dn = predict_LB_norm(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_LB_Dn = predict_LB_norm(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_LB_Dn = predict_LB_norm(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# # compute average log-likelihood for LB
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_LB_Dn)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_LB_Dn)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_LB_Dn)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_LB_Dn)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
#compute ECE for LB
print(scoring.expected_calibration_error(targets, MNIST_test_in_LB_Dn))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_LB_Dn))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_LB_Dn))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_LB_Dn))

In [ ]:
acc_in_LB_D, prob_correct_in_LB_D, ent_in_LB_D, MMC_in_LB_D = get_in_dist_values(MNIST_test_in_LB_Dn, targets)
acc_out_FMNIST_LB_D, prob_correct_out_FMNIST_LB_D, ent_out_FMNIST_LB_D, MMC_out_FMNIST_LB_D, auroc_out_FMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_Dn, MNIST_test_out_FMNIST_LB_Dn, targets_FMNIST)
acc_out_notMNIST_LB_D, prob_correct_out_notMNIST_LB_D, ent_out_notMNIST_LB_D, MMC_out_notMNIST_LB_D, auroc_out_notMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_Dn, MNIST_test_out_notMNIST_LB_Dn, targets_notMNIST)
acc_out_KMNIST_LB_D, prob_correct_out_KMNIST_LB_D, ent_out_KMNIST_LB_D, MMC_out_KMNIST_LB_D, auroc_out_KMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_Dn, MNIST_test_out_KMNIST_LB_Dn, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_LB_D, prob_correct_in_LB_D, ent_in_LB_D, MMC_in_LB_D, 'MNIST', 'LB_D')
print_out_dist_values(acc_out_FMNIST_LB_D, prob_correct_out_FMNIST_LB_D, ent_out_FMNIST_LB_D, MMC_out_FMNIST_LB_D, auroc_out_FMNIST_LB_D, 'MNIST', test='fmnist', method='LB_D')
print_out_dist_values(acc_out_notMNIST_LB_D, prob_correct_out_notMNIST_LB_D, ent_out_notMNIST_LB_D, MMC_out_notMNIST_LB_D, auroc_out_notMNIST_LB_D, 'MNIST', test='notMNIST', method='LB_D')
print_out_dist_values(acc_out_KMNIST_LB_D, prob_correct_out_KMNIST_LB_D, ent_out_KMNIST_LB_D, MMC_out_KMNIST_LB_D, auroc_out_KMNIST_LB_D, 'MNIST', test='KMNIST', method='LB_D')

# KFAC Laplace Bridge

In [ ]:
MNIST_test_in_LB_KFAC = predict_LB(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_LB_KFAC = predict_LB(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_LB_KFAC = predict_LB(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_LB_KFAC = predict_LB(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for LB KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_LB_KFAC)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_LB_KFAC)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_LB_KFAC)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_LB_KFAC)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_LB_KFAC))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_LB_KFAC))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_LB_KFAC))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_LB_KFAC))

In [ ]:
acc_in_LB_KFAC, prob_correct_in_LB_KFAC, ent_in_LB_KFAC, MMC_in_LB_KFAC = get_in_dist_values(MNIST_test_in_LB_KFAC, targets)
acc_out_FMNIST_LB_KFAC, prob_correct_out_FMNIST_LB_KFAC, ent_out_FMNIST_LB_KFAC, MMC_out_FMNIST_LB_KFAC, auroc_out_FMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFAC, MNIST_test_out_FMNIST_LB_KFAC, targets_FMNIST)
acc_out_notMNIST_LB_KFAC, prob_correct_out_notMNIST_LB_KFAC, ent_out_notMNIST_LB_KFAC, MMC_out_notMNIST_LB_KFAC, auroc_out_notMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFAC, MNIST_test_out_notMNIST_LB_KFAC, targets_notMNIST)
acc_out_KMNIST_LB_KFAC, prob_correct_out_KMNIST_LB_KFAC, ent_out_KMNIST_LB_KFAC, MMC_out_KMNIST_LB_KFAC, auroc_out_KMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFAC, MNIST_test_out_KMNIST_LB_KFAC, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_LB_KFAC, prob_correct_in_LB_KFAC, ent_in_LB_KFAC, MMC_in_LB_KFAC, 'MNIST', 'LB_KFAC')
print_out_dist_values(acc_out_FMNIST_LB_KFAC, prob_correct_out_FMNIST_LB_KFAC, ent_out_FMNIST_LB_KFAC, MMC_out_FMNIST_LB_KFAC, auroc_out_FMNIST_LB_KFAC, 'MNIST', test='fmnist', method='LB_KFAC')
print_out_dist_values(acc_out_notMNIST_LB_KFAC, prob_correct_out_notMNIST_LB_KFAC, ent_out_notMNIST_LB_KFAC, MMC_out_notMNIST_LB_KFAC, auroc_out_notMNIST_LB_KFAC, 'MNIST', test='notMNIST', method='LB_KFAC')
print_out_dist_values(acc_out_KMNIST_LB_KFAC, prob_correct_out_KMNIST_LB_KFAC, ent_out_KMNIST_LB_KFAC, MMC_out_KMNIST_LB_KFAC, auroc_out_KMNIST_LB_KFAC, 'MNIST', test='KMNIST', method='LB_KFAC')

# LB kfac norm

In [ ]:
MNIST_test_in_LB_KFACn = predict_LB_norm(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_LB_KFACn = predict_LB_norm(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_LB_KFACn = predict_LB_norm(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_LB_KFACn = predict_LB_norm(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for LB KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_LB_KFACn)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_LB_KFACn)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_LB_KFACn)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_LB_KFACn)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_LB_KFACn))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_LB_KFACn))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_LB_KFACn))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_LB_KFACn))

In [ ]:
acc_in_LB_KFAC, prob_correct_in_LB_KFAC, ent_in_LB_KFAC, MMC_in_LB_KFAC = get_in_dist_values(MNIST_test_in_LB_KFACn, targets)
acc_out_FMNIST_LB_KFAC, prob_correct_out_FMNIST_LB_KFAC, ent_out_FMNIST_LB_KFAC, MMC_out_FMNIST_LB_KFAC, auroc_out_FMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFACn, MNIST_test_out_FMNIST_LB_KFACn, targets_FMNIST)
acc_out_notMNIST_LB_KFAC, prob_correct_out_notMNIST_LB_KFAC, ent_out_notMNIST_LB_KFAC, MMC_out_notMNIST_LB_KFAC, auroc_out_notMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFACn, MNIST_test_out_notMNIST_LB_KFACn, targets_notMNIST)
acc_out_KMNIST_LB_KFAC, prob_correct_out_KMNIST_LB_KFAC, ent_out_KMNIST_LB_KFAC, MMC_out_KMNIST_LB_KFAC, auroc_out_KMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFACn, MNIST_test_out_KMNIST_LB_KFACn, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_LB_KFAC, prob_correct_in_LB_KFAC, ent_in_LB_KFAC, MMC_in_LB_KFAC, 'MNIST', 'LB_KFAC')
print_out_dist_values(acc_out_FMNIST_LB_KFAC, prob_correct_out_FMNIST_LB_KFAC, ent_out_FMNIST_LB_KFAC, MMC_out_FMNIST_LB_KFAC, auroc_out_FMNIST_LB_KFAC, 'MNIST', test='fmnist', method='LB_KFAC')
print_out_dist_values(acc_out_notMNIST_LB_KFAC, prob_correct_out_notMNIST_LB_KFAC, ent_out_notMNIST_LB_KFAC, MMC_out_notMNIST_LB_KFAC, auroc_out_notMNIST_LB_KFAC, 'MNIST', test='notMNIST', method='LB_KFAC')
print_out_dist_values(acc_out_KMNIST_LB_KFAC, prob_correct_out_KMNIST_LB_KFAC, ent_out_KMNIST_LB_KFAC, MMC_out_KMNIST_LB_KFAC, auroc_out_KMNIST_LB_KFAC, 'MNIST', test='KMNIST', method='LB_KFAC')

# Compare to probit (extended MacKay approach)

In [ ]:
MNIST_test_in_PROBIT_D = predict_probit(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_PROBIT_D = predict_probit(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_PROBIT_D = predict_probit(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_PROBIT_D = predict_probit(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for LB KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_PROBIT_D)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_PROBIT_D)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_PROBIT_D)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_PROBIT_D)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_PROBIT_D))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_PROBIT_D))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_PROBIT_D))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_PROBIT_D))

In [ ]:
acc_in_PROBIT, prob_correct_in_PROBIT, ent_in_PROBIT, MMC_in_PROBIT = get_in_dist_values(MNIST_test_in_PROBIT_D, targets)
acc_out_FMNIST_PROBIT, prob_correct_out_FMNIST_PROBIT, ent_out_FMNIST_PROBIT, MMC_out_FMNIST_PROBIT, auroc_out_FMNIST_PROBIT = get_out_dist_values(MNIST_test_in_PROBIT_D, MNIST_test_out_FMNIST_PROBIT_D, targets_FMNIST)
acc_out_notMNIST_PROBIT, prob_correct_out_notMNIST_PROBIT, ent_out_notMNIST_PROBIT, MMC_out_notMNIST_PROBIT, auroc_out_notMNIST_PROBIT = get_out_dist_values(MNIST_test_in_PROBIT_D, MNIST_test_out_notMNIST_PROBIT_D, targets_notMNIST)
acc_out_KMNIST_PROBIT, prob_correct_out_KMNIST_PROBIT, ent_out_KMNIST_PROBIT, MMC_out_KMNIST_PROBIT, auroc_out_KMNIST_PROBIT = get_out_dist_values(MNIST_test_in_PROBIT_D, MNIST_test_out_KMNIST_PROBIT_D, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_PROBIT, prob_correct_in_PROBIT, ent_in_PROBIT, MMC_in_PROBIT, 'MNIST', 'PROBIT')
print_out_dist_values(acc_out_FMNIST_PROBIT, prob_correct_out_FMNIST_PROBIT, ent_out_FMNIST_PROBIT, MMC_out_FMNIST_PROBIT, auroc_out_FMNIST_PROBIT, 'MNIST', test='fmnist', method='PROBIT')
print_out_dist_values(acc_out_notMNIST_PROBIT, prob_correct_out_notMNIST_PROBIT, ent_out_notMNIST_PROBIT, MMC_out_notMNIST_PROBIT, auroc_out_notMNIST_PROBIT, 'MNIST', test='notMNIST', method='PROBIT')
print_out_dist_values(acc_out_KMNIST_PROBIT, prob_correct_out_KMNIST_PROBIT, ent_out_KMNIST_PROBIT, MMC_out_KMNIST_PROBIT, auroc_out_KMNIST_PROBIT, 'MNIST', test='KMNIST', method='PROBIT')

### PROBIT KFAC

In [ ]:
MNIST_test_in_PROBIT_K = predict_probit(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_PROBIT_K = predict_probit(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_PROBIT_K = predict_probit(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_PROBIT_K = predict_probit(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for LB KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_PROBIT_K)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_PROBIT_K)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_PROBIT_K)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_PROBIT_K)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_PROBIT_K))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_PROBIT_K))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_PROBIT_K))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_PROBIT_K))

In [ ]:
acc_in_PROBIT, prob_correct_in_PROBIT, ent_in_PROBIT, MMC_in_PROBIT = get_in_dist_values(MNIST_test_in_PROBIT_K, targets)
acc_out_FMNIST_PROBIT, prob_correct_out_FMNIST_PROBIT, ent_out_FMNIST_PROBIT, MMC_out_FMNIST_PROBIT, auroc_out_FMNIST_PROBIT = get_out_dist_values(MNIST_test_in_PROBIT_K, MNIST_test_out_FMNIST_PROBIT_K, targets_FMNIST)
acc_out_notMNIST_PROBIT, prob_correct_out_notMNIST_PROBIT, ent_out_notMNIST_PROBIT, MMC_out_notMNIST_PROBIT, auroc_out_notMNIST_PROBIT = get_out_dist_values(MNIST_test_in_PROBIT_K, MNIST_test_out_notMNIST_PROBIT_K, targets_notMNIST)
acc_out_KMNIST_PROBIT, prob_correct_out_KMNIST_PROBIT, ent_out_KMNIST_PROBIT, MMC_out_KMNIST_PROBIT, auroc_out_KMNIST_PROBIT = get_out_dist_values(MNIST_test_in_PROBIT_K, MNIST_test_out_KMNIST_PROBIT_K, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_PROBIT, prob_correct_in_PROBIT, ent_in_PROBIT, MMC_in_PROBIT, 'MNIST', 'PROBIT')
print_out_dist_values(acc_out_FMNIST_PROBIT, prob_correct_out_FMNIST_PROBIT, ent_out_FMNIST_PROBIT, MMC_out_FMNIST_PROBIT, auroc_out_FMNIST_PROBIT, 'MNIST', test='fmnist', method='PROBIT')
print_out_dist_values(acc_out_notMNIST_PROBIT, prob_correct_out_notMNIST_PROBIT, ent_out_notMNIST_PROBIT, MMC_out_notMNIST_PROBIT, auroc_out_notMNIST_PROBIT, 'MNIST', test='notMNIST', method='PROBIT')
print_out_dist_values(acc_out_KMNIST_PROBIT, prob_correct_out_KMNIST_PROBIT, ent_out_KMNIST_PROBIT, MMC_out_KMNIST_PROBIT, auroc_out_KMNIST_PROBIT, 'MNIST', test='KMNIST', method='PROBIT')

In [ ]:
break

# Compare to Second-order Delta Posterior Predictive

this kinda sucks. So we'll just leave it alone.

In [ ]:
MNIST_test_in_SODPP_D = predict_second_order_dpp(la_diag, MNIST_test_loader, timing=True, device=device).cpu().nan_to_num(nan=0.0, neginf=0.0, posinf=1.0).numpy()
MNIST_test_out_FMNIST_SODPP_D = predict_second_order_dpp(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().nan_to_num(nan=0.0, neginf=0.0, posinf=1.0).numpy()
MNIST_test_out_notMNIST_SODPP_D = predict_second_order_dpp(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().nan_to_num(nan=0.0, neginf=0.0, posinf=1.0).numpy()
MNIST_test_out_KMNIST_SODPP_D = predict_second_order_dpp(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().nan_to_num(nan=0.0, neginf=0.0, posinf=1.0).numpy()

In [ ]:
# compute average log-likelihood for LB KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_SODPP_D)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_SODPP_D)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_SODPP_D)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_SODPP_D)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_SODPP_D))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_SODPP_D))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_SODPP_D))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_SODPP_D))

In [ ]:
acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP = get_in_dist_values(MNIST_test_in_SODPP_D, targets)
acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_D, MNIST_test_out_FMNIST_SODPP_D, targets_FMNIST)
acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_D, MNIST_test_out_notMNIST_SODPP_D, targets_notMNIST)
acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_D, MNIST_test_out_KMNIST_SODPP_D, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP, 'MNIST', 'SODPP')
print_out_dist_values(acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP, 'MNIST', test='fmnist', method='SODPP')
print_out_dist_values(acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP, 'MNIST', test='notMNIST', method='SODPP')
print_out_dist_values(acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP, 'MNIST', test='KMNIST', method='SODPP')

### KFAC SODPP

In [ ]:
MNIST_test_in_SODPP_K = predict_second_order_dpp(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_SODPP_K = predict_second_order_dpp(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_SODPP_K = predict_second_order_dpp(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_SODPP_K = predict_second_order_dpp(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
# compute average log-likelihood for LB KFAC
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_in_SODPP_K)).log_prob(torch.tensor(targets)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_SODPP_K)).log_prob(torch.tensor(targets_FMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_SODPP_K)).log_prob(torch.tensor(targets_notMNIST)).mean().item())
print(-torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_SODPP_K)).log_prob(torch.tensor(targets_KMNIST)).mean().item())

In [ ]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_SODPP_K))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_SODPP_K))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_SODPP_K))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_SODPP_K))

In [ ]:
acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP = get_in_dist_values(MNIST_test_in_SODPP_K, targets)
acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_K, MNIST_test_out_FMNIST_SODPP_K, targets_FMNIST)
acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_K, MNIST_test_out_notMNIST_SODPP_K, targets_notMNIST)
acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_K, MNIST_test_out_KMNIST_SODPP_K, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP, 'MNIST', 'SODPP')
print_out_dist_values(acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP, 'MNIST', test='fmnist', method='SODPP')
print_out_dist_values(acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP, 'MNIST', test='notMNIST', method='SODPP')
print_out_dist_values(acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP, 'MNIST', test='KMNIST', method='SODPP')